In [1]:
# This notebook handles data processing for the Faraday Rotation lab

# Imports
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

In [1]:
# Useful functions

# Reads a data file, returning a dictionary of its data
def read_file(file: str, axes: tuple) -> list[dict, dict]:
    
    # The data read
    data = {}
    
    # The uncertainty associated with the data
    uncs = {}
    
    # Initialises data dictionaries
    for axis in axes:
        data[axis] = []
        uncs[axis] = None
    
    
    # Reads the file
    with open(file + '.txt', "r") as file:
        
        # Tracks line number
        # Line 0 is metadata
        # Line 1 are the uncertainties
        # Lines 2+ is the data
        i = 0
        for line in file:
            
            # Deliminates each line with a comma
            split = line.split(',')
            
            # Handles the uncertainties
            if i == 1:
                for j in range(1, len(axes)):
                    uncs[axes[j]] = float(split[j])
            
            # Third line is the start of the data
            elif i > 1:
                for j in range(len(axes)):
                    data[axes[j]].append(float(split[j]))
                
            # Increments to next line
            i += 1
            
    # Returns
    return data, uncs


# A Plotter holds data that it can plot
class Plotter:
    def __init__(self, file, axes, save = False):
        self.data, self.uncs = read_file(file, axes)
        self.slope = None
        self.intercept = None
        
        self.save = False
        
    # Calling the plotter plots some data
    def __call__(self, x_axis, y_axis, x_label, y_label, title, fit = True):
        
        # Obtains the figure components
        fig, ax = plt.subplots()

        # Obtains the x-values
        x = self.data[x_axis]
        x_unc = self.uncs[x_axis]

        # Obtains the y-values
        y = self.data[y_axis]
        y_unc = self.uncs[y_axis]

        # Plots the data
        ax.scatter(x, y)    
        
        # Plots the errors
        #ax.errorbar(x, y, xerr = x_unc, yerr = y_unc)

        # Adds axis titles
        #plt.xlabel(x_label)
        #plt.ylabel(y_label)
        plt.xlabel(x_axis)
        plt.ylabel(y_axis)
        
        # Adds a title
        plt.title(title)
        
        
        # Performs a linear fit (if desired)
        if fit:

            # Performs the linear fit
            a_fit, cov = curve_fit(linear_fit, x, y, sigma = y_unc)

            # Obtains the important parameters from the linear fit
            self.slope = a_fit[0]
            self.intercept = a_fit[1]

            # Obtains the uncertainty of the slope and intercept
            # Uncertainty is the square-root of the diagonal elements
            uncs = [np.sqrt(cov[i][i]) for i in range(len(cov))]

            # Creates 
            lin_x = x
            lin_y = [linear_fit(i, self.slope, self.intercept) for i in lin_x]
            
            ax.plot(lin_x, lin_y)

        # Shows (or saves) the graph
        if not self.save:
            plt.show()
        else:
            plt.savefig(name + '.png', bbox_inches='tight')
            
    # Returns data for the specified axes
    def get_axes(self, x_axis, y_axis):
        return self.data[x_axis], self.data[y_axis]
    
    # Transforms an axis, adding it to data
    # This is used to convert current to magnetic field
    def transform(self, axis, new_axis, new_unc, fnc, params):
        for i in range(len(self.data[axis])):
            new_column = fnc(self.data[axis], params)
            
        # Adds the transformed data
        self.add_axis(new_axis, new_column, new_unc)
        
    # Adds a new axis/column to data
    def add_axis(self, new_axis, new_column, new_unc):
        self.data[new_axis] = new_column
        self.uncs[new_axis] = new_unc
            
            
# Our linear fitting function
def linear_fit(x, m, b):
    return m * x + b

# Transforms voltage reading to magnetic field
def voltage_to_magnetic(column, params):
    new_column = []
    
    for voltage in column:
        B = (voltage - params['V_Q']) / params['T_term']
        new_column.append(B)
        
    return new_column

# Gets the Verdet constant
def verdet_manual(plotter, L, mag_conversion, degrees = True):
    if not degrees:
        slope = plotter.slope / L / mag_conversion * np.pi / 180
    return plotter.slope / L / mag_conversion

def verdet_lockin(plotter, L, mag_conversion, degrees = True):
    return verdet_manual(plotter, L, mag_conversion, degrees) / 2 * np.sqrt(2) / plotter.data['I_0'][0]





# Returns verdet constant
def n_verdet(plotter, L, c):
    return plotter.slope / L / plotter.data['I_0'][0] * mag_plotter.data['teslas'][0]

# Converts current to magnetic field
def B_0(I):
    return mag_plotter.data['teslas'][0] / 2 * I

In [2]:
# Reads data

# Defines the data files available
files = ['day1', 'day2']
mag_files = ['mag']
red_files = ['ac1', 'ac2']
green_files = ['green_water', 'green_water2', 'green_rod']

# Prepares a plotter with the data
manual_plotters = {
    'm0': Plotter(files[0], ['amps', 'volts', 'degrees']),
    'm1': Plotter(files[1], ['amps', 'volts', 'degrees']),
}

lockin_plotters = {
    'red_glass': Plotter(red_files[0], ['amps', 'I_0', '∆I']),
    'red_water': Plotter(red_files[1], ['amps', 'I_0', '∆I']),
    'green_rod': Plotter(green_files[2], ['amps', 'I_0', '∆I']),
    'green_water0': Plotter(green_files[0], ['amps', 'I_0', '∆I']),
    'green_water1': Plotter(green_files[1], ['amps', 'I_0', '∆I']),
    
}

mag_plotter = Plotter(mag_files[0], ['position', 'volts'])

#plotter0 = Plotter(files[0], ['amps', 'volts', 'degrees'])
#plotter1 = Plotter(files[1], ['amps', 'volts', 'degrees'])

#red_glass = Plotter(red_files[0], ['amps', 'I_0', '∆I'])
#red_water = Plotter(red_files[1], ['amps', 'I_0', '∆I'])
#green_water = Plotter(green_files[0], ['amps', 'I_0', '∆I'])
#green_water2 = Plotter(green_files[1], ['amps', 'I_0', '∆I'])
#green_rod = Plotter(green_files[2], ['amps', 'I_0', '∆I'])


In [3]:
# Does some calculations

S = 50
S_TC = 0.12
T_A = 40.5

# Prepares the transform
transform_params = {
    'V_Q': 2.5,
    'dV_Q': 0.7,
    'T_term': S * (1 + S_TC * (T_A - 25))
}

mag_plotter.transform('volts', 'teslas', [0 for i in range(len(list(mag_plotter.data.values())[0]))], voltage_to_magnetic, transform_params)

In [4]:
# Plots our data

#print(f'Slope:\t\t{a_fit[0]} ± {uncs[0]} [1/V]\nIntercept:\t{a_fit[1]} ± {uncs[1]} [unitless]')
#plotter0('amps', 'degrees', 'Current Supplied to Solenoid (A)', 'Rotation of Light (degrees)', 'plotter0', slope = a_fit[0], intercept = a_fit[1])
#plotter0('amps', 'degrees', 'Current Supplied to Solenoid (A)', 'Rotation of Light (degrees)', 'Red Laser & Glass (Exctinction Method)')
#plotter1('amps', 'volts', 'Current Supplied to Solenoid (A)', 'Phototransistor Voltage at Measurement (V)', 'plotter1')
#plotter1('amps', 'degrees', 'Current Supplied to Solenoid (A)', 'Rotation of Light (degrees)', 'Red Laser & Glass (45º Method)')

#mag_plotter('position', 'volts', 'Position of Hall prombe (cm)', 'Voltage from Hall probe (V)', 'Hall Probe', fit = False)
mag_plotter('position', 'teslas', 'Position of Hall prombe (cm)', 'Teslas (T)', 'Hall Probe (Transformed)', fit = False)

#red_glass('amps', '∆I', 'a', 'b', 'Red Laser & Glass')
#red_water('amps', '∆I', 'a', 'b', 'Red Laser & Water')
#green_water('amps', '∆I', 'a', 'b', 'Green Laser & Water (1)')
#green_water2('amps', '∆I', 'a', 'b', 'Green Laser & Water (2)')
#green_rod('amps', '∆I', 'a', 'b', 'Green Laser & Rod')

for plotter in manual_plotters:
    manual_plotters[plotter]('amps', 'degrees', 'Current Supplied to Solenoid (A)', 'Rotation of Light (degrees)', plotter)
    
for plotter in lockin_plotters:
    lockin_plotters[plotter]('amps', '∆I', 'Current Supplied to Solenoid (A)', 'RMS Intensity', plotter)

NameError: name 'plt' is not defined

In [61]:
# Solves for the Verdat constant
# theta = VBL -> V = theta / VL
# t_0 = 1/2 * ∆I / I_0

# We measured the magnetic field at 2 Amps, so this is our conversion
mag_conversion = 2 / mag_plotter.data['teslas'][0]

# Calculates the verdet constant for each trial
verdets = {}

for plotter in manual_plotters:
    verdets[plotter] = verdet_manual(manual_plotters[plotter], 0.10, mag_conversion)
    
for plotter in lockin_plotters:
    verdets[plotter] = n_verdet(lockin_plotters[plotter], 0.10, 1)
#    v = []
#    for point in range(len(list(lockin_plotters[plotter].data.values())[0])):
#        v.append(lockin_plotters[plotter].data.values['∆I'][point] / lockin_plotters[plotter].data.values['I_0'][point] / 2)
        
#print(v)


    
# Shows each verdet constant
for constant in verdets:
    print(f'{constant}:\t{verdets[constant]}')


m0:	0.04790997462615002
m1:	0.0426796285293022
red_glass:	3.0338002132072743
red_water:	0.6718916364798593
green_rod:	4.809716595563665
green_water0:	2.4310087125124356
green_water1:	3.184650454415512
